# Otimização do Serviço de Ônibus para Embarque Remoto

Este notebook apresenta a **formulação matemática** e a **implementação da resolução** do problema de otimização considerado.

## Descrição do Problema

O problema consiste em definir as rotas dos ônibus utilizados para levar passageiros do portão de embarque até o avião ou do avião até o portão de embarque. Essa rota deve minimizar os custos de transporte que, nesse caso, será interpretado como equivalente à distância total percorrida. Os ônibus têm capacidades iguais e cada grupo de passageiros de cada voo será desmembrado em requisições. Logo, o modelo será escrito para determinar as sequência de requisições que cada ônibus deve fazer.

### Características Principais:
- **Janelas de tempo** específicas para cada requisição
- **Frota limitada** de ônibus
- **Restrições de autonomia** (distância máxima por viagem)
- **Múltiplas viagens** por ônibus durante o período de operação

## 1. Importação de Bibliotecas e Carregamento dos Dados

Iniciamos importando as bibliotecas necessárias e carregando uma instância do problema:

In [1]:
import gurobipy as gp
from gurobipy import GRB
from time import time
from dados import carrega_dados_json
from solucao import Solucao

nome_arquivo = "./dados/pequena.json"
dados = carrega_dados_json(nome_arquivo)

modelo = gp.Model("Otimização do Serviço de Ônibus para Embarque Remoto")

Set parameter Username
Set parameter LicenseID to value 2698967
Academic license - for non-commercial use only - expires 2026-08-22


## 2. Definição dos Conjuntos e Constantes

O problema é modelado como um **Problema de Programação Linear Inteira Mista** similar ao problema de roteamento de veículos com coleta e entrega acoplados, janelas de tempo, múltiplas viagens e distância máxima. O problema possui as seguintes características:

### Conjuntos:
- **N**: Conjunto de requisições {1, 2, ..., n}
- **N₀**: Conjunto de requisições incluindo a garagem {0, 1, 2, ..., n}
- **K**: Conjunto de ônibus {1, 2, ..., k}
- **V**: Conjunto de viagens possíveis por ônibus {1, 2, ..., r}

### Parâmetros do Modelo:

- **D[i,j]**: Distância entre as requisições i e j
- **c[i,j]**: Custo de viagem entre as requisições i e j  
- **T[i,j]**: Tempo de viagem entre as requisições i e j
- **s[i]**: Tempo de serviço na requisição i
- **e[i], l[i]**: Início e fim da janela de tempo da requisição i
- **Tmax**: Tempo máximo de duração de uma viagem

In [2]:
N = list(range(1, dados.n+1))
N0 = list(range(dados.n+1))
V = list(range(1, dados.r+1))
K = list(range(1, dados.K+1))
x = {}
y = {}
B = {}
M = 1e7
LIMITE_TEMPO = 5*60.

## 3. Variáveis de Decisão

O modelo utiliza três tipos de variáveis de decisão:

### **Variáveis Binárias:**

**x[i,j,v,k]**: Variável binária que indica se o ônibus k na viagem v percorre o arco de i para j
- **x[i,j,v,k] = 1**: O ônibus k vai da requisição i para j na viagem v
- **x[i,j,v,k] = 0**: Caso contrário

**y[v,k]**: Variável binária que indica se o ônibus k realiza a viagem v
- **y[v,k] = 1**: O ônibus k executa a viagem v
- **y[v,k] = 0**: Caso contrário

### **Variáveis Contínuas:**

**B[i,v,k]**: Tempo de chegada do ônibus k na requisição i durante a viagem v

In [3]:
for k in K:
    for v in V:
        for i in N0:
            for j in N0:
                if i != j:
                    x[i, j, v, k] = modelo.addVar(vtype=GRB.BINARY,
                                            name=f"x_{i}_{j}_{v}_{k}")

            B[i, v, k] = modelo.addVar(vtype=GRB.CONTINUOUS,
                                        lb=0.0,
                                        name=f"B_{i}_{v}_{k}")
            
        y[v, k] = modelo.addVar(vtype=GRB.BINARY,
                                name=f"y_{v}_{k}")
modelo.update()

## 4. Função Objetivo

**Minimizar o custo total de operação dos ônibus:**

$$\min \sum_{i \in N_0} \sum_{j \in N_0} \sum_{v \in V} \sum_{k \in K} c_{ij} \cdot x_{ijvk}$$

onde:
- **c[i,j]**: Custo de percorrer o arco da requisição i para j
- **x[i,j,v,k]**: Variável binária de roteamento

O objetivo é **minimizar a soma de todos os custos** dos arcos utilizados por todos os ônibus em todas as viagens.

In [4]:
funcao_objetivo = modelo.setObjective(
    gp.quicksum(dados.c[i, j] * x[i, j, v, k] 
                for i in N0 
                for j in N0
                for v in V
                for k in K
                if i != j),
    GRB.MINIMIZE
)

## 5. Restrições do Modelo

### 5.1. Restrição de Atendimento

**Cada requisição deve ser atendida exatamente uma vez:**

$$\sum_{i \in N_0} \sum_{k \in K} \sum_{v \in V} x_{ijvk} = 1, \quad \forall j \in N$$

Esta restrição garante que:
- **Toda requisição é atendida** 
- **Nenhuma requisição é atendida mais de uma vez**
- **Não há requisições não atendidas**

In [5]:
for j in N:
    modelo.addConstr(
        gp.quicksum(x[i, j, v, k]
                    for i in N0
                    for k in K
                    for v in V
                    if i != j) == 1,
        name=f"atendimento_{j}"
)

### 5.2. Restrições de Conservação de Fluxo

**O que entra deve sair - continuidade das rotas:**

$$\sum_{i \in N_0, i \neq j} x_{ijvk} - \sum_{i \in N_0, i \neq j} x_{jivk} = 0, \quad \forall j \in N, \forall v \in V, \forall k \in K$$

Esta restrição garante que:
- **Se um ônibus chega** a uma requisição j, **ele deve sair** dela
- **Continuidade das rotas** sem interrupções
- **Estrutura de caminho** válida no grafo

In [6]:
for k in K:
    for v in V:
        for j in N:

            entrada = gp.quicksum(x[i, j, v, k]
                                for i in N0
                                if i != j)

            saida = gp.quicksum(x[j, i, v, k]
                                for i in N0
                                if i != j)

            modelo.addConstr(
                entrada - saida == 0,
                name=f"conservacao_{j}_{v}_{k}"
            )


### 5.3. Restrições de Início e Término de Viagens

**Toda viagem começa e termina na garagem:**

**Início da viagem:**
$$\sum_{j \in N} x_{0jvk} = y_{vk}, \quad \forall v \in V, \forall k \in K$$

**Término da viagem:**
$$\sum_{i \in N} x_{i0vk} = y_{vk}, \quad \forall v \in V, \forall k \in K$$

Estas restrições garantem que:
- **Toda viagem ativa** sai da garagem (nó 0)
- **Toda viagem ativa** retorna à garagem
- **Viagens inativas** não têm movimento
- **Consistência** entre variáveis x e y

In [7]:
for k in K:
    for v in V:
        modelo.addConstr(
            gp.quicksum(x[0, j, v, k]
                        for j in N) == y[v, k],
            name=f"inicio_viagem_{v}_{k}"
        )
        modelo.addConstr(
            gp.quicksum(x[i, 0, v, k]
                        for i in N) == y[v, k],
            name=f"termino_viagem_{v}_{k}"
        )

### 5.4. Restrições de Sequenciamento de Viagens

**As viagens devem ser executadas em ordem sequencial:**

$$y_{vk} \leq y_{(v-1)k}, \quad \forall v \in V, v > 1, \forall k \in K$$

Esta restrição garante que:
- **Viagem v só pode ser ativa** se a viagem (v-1) também for ativa
- **Ordem cronológica** das viagens
- **Não há gaps** entre viagens (ex: viagem 1 e 3 ativas, mas 2 inativa)
- **Estrutura sequencial** lógica

In [8]:
for k in K:
    for v in V:
        if v > 1:
            modelo.addConstr(
                y[v, k] <= y[v-1, k],
                name=f"sequencia_viagem_{v}_{k}"
            )

### 5.5. Restrições de Janelas de Tempo

**O atendimento deve ocorrer dentro do período especificado:**

**Limite inferior:**
$$B_{ivk} \geq e_i \cdot \sum_{j \in N_0, j \neq i} x_{jivk}, \quad \forall i \in N, \forall v \in V, \forall k \in K$$

**Limite superior:**
$$B_{ivk} \leq l_i \cdot \sum_{j \in N_0, j \neq i} x_{jivk}, \quad \forall i \in N, \forall v \in V, \forall k \in K$$

Estas restrições garantem que:
- **Atendimento não antecipado**: B[i,v,k] ≥ e[i]
- **Atendimento não atrasado**: B[i,v,k] ≤ l[i]  
- **Respeito aos horários** dos voos
- **Coordenação temporal** das operações

In [9]:
for k in K:
    for v in V:
        for i in N:
            modelo.addConstr(
                B[i, v, k] >= dados.e[i-1] * gp.quicksum(x[j, i, v, k]
                for j in N0 if i != j),
                name=f"janela_inferior_{i}_{v}_{k}"
            )
            modelo.addConstr(
                B[i, v, k] <= dados.l[i-1] * gp.quicksum(x[j, i, v, k]
                for j in N0 if i != j),
                name=f"janela_superior_{i}_{v}_{k}"
            )

### 5.6. Restrições de Fluxo de Tempo

**Coordenação temporal entre requisições e viagens:**

#### **Fluxo Intra-viagem** (dentro da mesma viagem):
$$B_{ivk} + s_i + T_{ij} - M \cdot (1 - x_{ijvk}) \leq B_{jvk}, \quad \forall i \in N, j \in N_0, i \neq j, v \in V, k \in K $$

No caso da primeira requisição na primeira viagem de cada ônibus, a equação precisa ser ajustada para:

$$ s_0 + T_{0i} - M (1-x_{0i1k}) \le B_{i1k}, \quad \forall i \in N, k \in K $$

#### **Fluxo Inter-viagem** (entre viagens consecutivas):
$$B_{0(v-1)k} + s_0 + T_{0i} - M \cdot (1 - x_{0ivk}) \leq B_{ivk}, \quad \forall i \in N, v \in V, v > 1, k \in K$$

Estas restrições garantem que:
- **Sequência temporal** correta dentro de cada viagem
- **Tempo de serviço** é considerado em cada requisição
- **Tempo de deslocamento** entre requisições é respeitado
- **Continuidade temporal** entre viagens do mesmo ônibus
- **Tempo de reabastecimento** na garagem entre viagens

Onde **M** é uma constante suficientemente grande (Big-M) para tornar a restrição inativa quando x=0.

In [10]:
for k in K:
    for v in V:
        for i in N0:
            for j in N0:
                
                if i == j:
                    continue
                
                elif i == 0 and v == 1:
                    modelo.addConstr(
                        dados.s[0] + dados.T[0, j] - M * (1 - x[0, j, 1, k]) 
                        <= B[j, 1, k],
                        name=f"fluxo_tempo_intra_0_{j}_{v}_{k}"
                    )

                elif i != 0:
                    modelo.addConstr(
                        B[i, v, k] + dados.s[i] + dados.T[i, j]
                        - M * (1 - x[i, j, v, k]) <= B[j, v, k],
                    name=f"fluxo_tempo_intra_{i}_{j}_{v}_{k}"
                )
            
            if v > 1 and i != 0:
                modelo.addConstr(
                    B[0, v-1, k] + dados.s[0] + dados.T[0, i] 
                    - M * (1 - x[0, i, v, k]) <= B[i, v, k],
                    name=f"fluxo_tempo_inter_{i}_{v}_{k}"
                )

### 5.7. Restrições de duração máxima de uma viagem

**Cada viagem deve durar no máximo uma quantidade de tempo ``T_max``.** O momento de início de uma viagem será considerado como instante no qual um ônibus sai da garagem para atender a primeira requisição numa viagem. Note que o ônibus pode ficar um certo tempo parado na garagem antes de iniciar o atendimento. Por isso, o ideal é calcularmos o instante de início de uma viagem como o instante em que ele chega à primeira requisição menos o tempo de trânsito até lá e menos o tempo de preparação. Para todas as viagens, a variável $B_{0vk}$ guarda o instante em que o ônibus chega na garagem ao fim de uma viagem. Por isso, para calcularmos o tempo total, basta realizarmos a diferença entre esses dois momentos, ou seja:

$$ B_{0vk} - B_{ivk} + s_0 + T_{0i} - M(1-x_{0ivk}) \le y_{vk} T^{max}, \quad i \in N, v \in V, k \in K$$

Esta restrição garante que:
- **Nenhuma viagem excede** o tempo máximo permitido

In [ ]:
for k in K:
    for i in N:
        modelo.addConstr(
            B[0, 1, k] - B[i, 1, k] + dados.s[0] + dados.T[0, i] - M*(1 - x[0, i, 1, k]) <= dados.Tmax * y[1, k],
            name=f"tempo_maximo_1_{k}"
        )

for k in K:
    for v in V:
        if v > 1:
            for i in N:
                modelo.addConstr(
                    B[0, v, k] - B[i, v, k] + dados.s[0] + dados.T[0, i] - M*(1 - x[0, i, v, k]) <= dados.Tmax * y[v, k],
                    name=f"tempo_viagem_{v}_{k}"
                )

## 6. Resolução do Modelo com Gurobi

Após definir todas as variáveis, função objetivo e restrições, executamos a otimização:

In [12]:
modelo.update()
modelo.setParam(GRB.Param.TimeLimit, LIMITE_TEMPO)
tic = time()
modelo.optimize()
tempo_total = time() - tic
if modelo.Status == GRB.OPTIMAL:
    print(f"Solução ótima encontrada em {tempo_total:.2f} segundos.")
elif modelo.Status == GRB.TIME_LIMIT:
    print(f"Tempo limite atingido em {tempo_total:.2f} segundos.")
else:
    print(f"Solução não encontrada. Status: {modelo.Status}")
    exit()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.3 LTS")

CPU model: Intel(R) Core(TM) i7-14700K, instruction set [SSE2|AVX|AVX2]
Thread count: 28 physical cores, 28 logical processors, using up to 28 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 5524 rows, 4812 columns and 36921 nonzeros
Model fingerprint: 0xe92262de
Variable types: 240 continuous, 4572 integer (4572 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [4e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 3959 rows and 2424 columns
Presolve time: 0.52s
Presolved: 1565 rows, 2388 columns, 26843 nonzeros
Variable types: 225 continuous, 2163 integer (2153 binary)

Root relaxation: objective 4.017486e+04, 832 iterations, 0.06 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 

## 7. Apresentação da Solução

A classe `Solucao` processa os resultados do Gurobi e apresenta:

In [13]:
solucao = Solucao()
solucao.carrega_modelo_gurobi(modelo, dados)
print(solucao) 

Ônibus 1: [0, 2, 3, 7, 0, 14, 0, 17, 0, 19, 0]
Ônibus 2: [0, 1, 4, 0, 10, 11, 0, 13, 0, 16, 0]
Ônibus 3: [0, 5, 6, 8, 0, 9, 12, 0, 15, 0, 18, 0]


In [14]:
# Imprimir todas as variáveis B com seus valores
print("Variáveis B (Tempo de chegada):")
print("=" * 50)

for k in K:
    print(f"\nÔnibus {k}:")
    for v in V:
        print(f"  Viagem {v}: {y[v,k].X}")
        for i in N0:
            valor = B[i, v, k].X
            if valor > 0.001:  # Mostra apenas valores significativos
                if i == 0:
                    print(f"    B[{i},{v},{k}] = {valor:.2f} (Garagem)")
                else:
                    print(f"    B[{i},{v},{k}] = {valor:.2f} (Requisição {i}, {dados.e[i-1]} - {dados.l[i-1]})")
        distancia_total = sum(dados.D[i, j] * x[i, j, v, k].X
                              for i in N0
                              for j in N0
                              if i != j)
        print(f"    Distância total da viagem {v} do ônibus {k}: {distancia_total:.2f} km")

Variáveis B (Tempo de chegada):

Ônibus 1:
  Viagem 1: 1.0
    B[0,1,1] = 84.67 (Garagem)
    B[2,1,1] = 15.51 (Requisição 2, 5 - 20)
    B[3,1,1] = 25.00 (Requisição 3, 10 - 25)
    B[7,1,1] = 53.00 (Requisição 7, 53 - 68)
    Distância total da viagem 1 do ônibus 1: 5951.00 km
  Viagem 2: 1.0
    B[0,2,1] = 198.45 (Garagem)
    B[14,2,1] = 192.00 (Requisição 14, 192 - 207)
    Distância total da viagem 2 do ônibus 1: 2100.79 km
  Viagem 3: 1.0
    B[0,3,1] = 220.53 (Garagem)
    B[17,3,1] = 212.10 (Requisição 17, 207 - 222)
    Distância total da viagem 3 do ônibus 1: 2100.79 km
  Viagem 4: 1.0
    B[0,4,1] = 239.88 (Garagem)
    B[19,4,1] = 232.00 (Requisição 19, 217 - 232)
    Distância total da viagem 4 do ônibus 1: 2100.79 km

Ônibus 2:
  Viagem 1: 1.0
    B[0,1,2] = 39.29 (Garagem)
    B[1,1,2] = 10.84 (Requisição 1, 0 - 15)
    B[4,1,2] = 30.00 (Requisição 4, 15 - 30)
    Distância total da viagem 1 do ônibus 2: 4658.76 km
  Viagem 2: 1.0
    B[0,2,2] = 157.56 (Garagem)
    B[1